In [1]:
import os
import cv2
import sys
import csv
import winsound
import csv
from PIL import Image
import csv
# import openpyxl as excel
import datetime
import pandas as pd


FilePath = r"E:\_C.elegans\2023-11-01\20231101-182842tdTomato\20231101181917_recover"

In [3]:
#timestamp_brightの読み込み
log_bright = FilePath + "/" + "log.csv"
csv_bright = open(log_bright,"r")
csvR_bright = csv.reader(csv_bright)
header = next(csvR_bright)

#timestamp_fluoreの読み込み
log_fluore = FilePath + "/" + "3Dtimestamp.csv"
csv_fluore = open(log_fluore,"r",encoding="utf-8")
csvR_fluore = csv.reader(csv_fluore)

#video_brightの読み込み
video_path = FilePath + "/" + "rec.avi"
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("読み込み失敗")
    sys.exit()

In [ ]:
def extract_frames_from_avi(input_filename):
    """_summary_
    aviファイルからフレームを抽出して画像ファイルとして保存する

    Args:
        input_filename (_type_): _description_
        output_folder (_type_): _description_
    """
    now = datetime.datetime.now()
    try:
        os.mkdir(os.getcwd()+ "/log/"+now.strftime("%Y%m%d%H%M%S"))
    except FileExistsError:
        pass
    filename = now.strftime(os.getcwd()+ "/log/" + "%Y%m%d%H%M%S/")


    cap = cv2.VideoCapture(input_filename)
    if not cap.isOpened():
        print("Error: Could not open video file.")
        return

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total frames in video: {frame_count}")

    if not os.path.exists(filename):
        os.makedirs(filename)

    frame_number = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_filename = os.path.join(filename, f"frame_{frame_number:04d}.jpg")
        cv2.imwrite(frame_filename, frame)

        frame_number += 1

    cap.release()
    cv2.destroyAllWindows()

##### Timing Matched Videoの生成

In [4]:
#video_matchingの保存先の作成
filename = "rec_matching.avi"
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
fps = 50
size = (640, 480)
writer = cv2.VideoWriter(FilePath+"/"+filename, fourcc, fps, size)

#リストの作成
mylist = []

i = 0

#matching listの作成
for row in csvR_fluore:
    time_fluore = row[2]
    print(i)
    i+=1

    for row in csvR_bright:
        time_bright = row[1]

        if float(time_fluore) <= float(time_bright):
            mylist.append(row[0])
            break
        else:
            pass

print(len(mylist))
# print(mylist)

n = 0
m = 0
while True:
    is_image, frame_img = cap.read()

    if is_image:
        #print(mylist[n])
        if n < len(mylist):
            if int(mylist[n]) == m:
                n += 1
                try:
                    writer.write(frame_img)
                except:
                    pass
            else:
                pass
        else:
            break
        m += 1
    else:
        break

cap.release()
writer.release()
csv_bright.close()
csv_fluore.close()

# 2
mylist_int = [int(x) for x in mylist]

output_csv_file = FilePath + "/" + "log_matching.csv"

with open(log_bright, 'r', newline='') as input_file, open(output_csv_file, 'w', newline='') as output_file:
    reader = csv.reader(input_file)
    writer = csv.writer(output_file)
    writer.writerow(["frame",
                    "timestamp",
                    "stageX[pulse]",
                    "stageY[pulse]",
                    "Δt[s]",
                    "Δd[μm]",
                    "velocity[μm/s]",
                    "detectionX[pixel]",
                    "detectionY[pixel]",
                    "gap_x[pixel]",
                    "gap_y[pixel]",
                    "gap_x[μm]",
                    "gap_y[μ]",
                    "gap_x[pulse]",
                    "gap_y[pulse]",
                    "error[μm]",
                    "kal_x[pixel]",
                    "kal_y[pixel]",
                    "radius[pixel]",
                    "asobi[pixel]"])

    for i, row in enumerate(reader):
        if i in mylist_int:
            writer.writerow(row)


print("edit finished")
# 処理の完了後に音を鳴らす
winsound.Beep(1000, 200)  # Beep(周波数, 持続時間 ミリ秒)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

##### csv

In [21]:
import csv
path = r"E:\_C.elegans\2024-03-28\20240328-174457tdTomato-good_1st\3Dtimestamp.csv"
# テキストファイルのパス
file_path = r"E:\_C.elegans\2024-04-18\20240418-161507tdTomato-30mW stop at the end\20240418161436\parameter.txt"



################################################
# parameter txt file "parameter.txt"
################################################
with open(file_path, 'r') as file:
    lines = file.readlines()

txt_data = {}
for line in lines:
    key, value = line.strip().split(':', 1)
    txt_data[key.strip()] = value.strip()

# start_time
start_time = float(txt_data['start_time'])

################################################
# Lightsheet csv file "3Dtimestamp.csv"
################################################
lightsheet_RawTime = []
lightsheet_time = []

with open(path, newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter='\t')
    for row in csvreader:
        lightsheet_RawTime.extend(map(float, row))
raw_Lenghth = len(lightsheet_RawTime)

for i in range(raw_Lenghth):
    if i == 0:
        pass
    else:
        if lightsheet_RawTime[i-1] > lightsheet_RawTime[i]:
            lightsheet_RawTime[i] = lightsheet_RawTime[i] + lightsheet_RawTime[i+1]/1000
            lightsheet_RawTime.pop(i+1)
            print(f"Detect Error at {i}th timestamp")
Lenghth = len(lightsheet_RawTime)

for i in range(Lenghth):
    if i == 0:
        lightsheet_time.append(start_time)
    else:
        lightsheet_time.append(lightsheet_time[i-1] + lightsheet_RawTime[i] - lightsheet_RawTime[i-1])




[58508.835053, 58508.855053, 58508.874053, 58508.894053, 58508.913053, 58508.933053, 58508.953053, 58508.972053, 58508.992053, 58509.012053, 58509.031053, 58509.051053, 58509.070053, 58509.090053, 58509.110053, 58509.129053, 58509.149053, 58509.169053, 58509.188053, 58509.208053, 58509.228053, 58509.247053, 58509.267053, 58509.286053, 58509.306053, 58509.326053, 58509.345053, 58509.365053, 58509.385053, 58509.404053, 58509.424053, 58509.443053, 58509.463053, 58509.483053, 58509.502053, 58509.522053, 58509.542053, 58509.561053, 58509.581053, 58509.600053, 58509.620053, 58509.640053, 58509.659053, 58509.679053, 58509.699053, 58509.718053, 58509.738053, 58509.758053, 58509.777053, 58509.797053, 58509.816053, 58509.836053, 58509.856053, 58509.875053, 58509.895053, 58509.915053, 58509.934053, 58509.954053, 58509.973053, 58509.993053, 58510.013053, 58510.032053, 58510.052053, 58510.072053, 58510.091053, 58510.111053, 58510.130053, 58510.150053, 58510.170053, 58510.189053, 58510.209053, 58510

##### マーカーを動画に書き込み

In [3]:

fps = 115
size = (640, 480)
radius = 20
asobi = 2
detection_x = 320
detection_y = 240
centerx = 269
centery = 240

video_path = r"C:\Users\y-yuk\Desktop\c.elegans_test\appropriate_fps\img\500 frames.avi"
csv_file = r"C:\Users\y-yuk\Desktop\c.elegans_test\appropriate_fps\img\Results.csv"
csvReader = pd.read_csv(csv_file,encoding="shift-jis")

#ファイル名、保存方法を指定して、write_detect(検出画像の保存先動画)を用意する
now = datetime.datetime.now()
filename = "Local.avi"
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
writer_detect = cv2.VideoWriter("./log/" + filename, fourcc, fps, size)

# 動画ファイル・座標ファイルの読み込み
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("読み込み失敗")
    sys.exit()

# 画像処理・動画保存
n = 0
while(cap.isOpened()):
    is_image, frame = cap.read()

    if is_image:
        #print(n)
        # if n < 30:
        detection_x = int(csvReader.iat[n,5])
        detection_y = int(csvReader.iat[n,6])

        # frame = cv2.circle(frame, (detection_x, detection_y), radius*2, (255, 0, 255))
        # rame = cv2.circle(frame, (detection_x, detection_y), asobi*2, (0, 255, 255))
        # frame = cv2.drawMarker(frame, (detection_x, detection_y), (0, 255, 125))
        # frame = cv2.drawMarker(frame, position=(centerx, centery), color=(255, 255, 0), markerType=cv2.MARKER_TILTED_CROSS, markerSize=10)
        frame = cv2.circle(frame, (detection_x, detection_y), 5, (0, 0, 255), -1)

        # 動画に現在の画像を書き込む-ver.detect
        try:
            writer_detect.write(frame)
        except:
            pass
    else:
        cap.release()
        writer_detect.release()
        print("video-detect finished")
        break

    n += 1


video-detect finished


##### AVI to MP4

In [ ]:
from moviepy.editor import VideoFileClip

def convert_mp4_to_avi(mp4_file, avi_file):
    # MP4形式の動画ファイルを読み込む
    video_clip = VideoFileClip(mp4_file)
    
    # AVI形式に変換して保存する
    video_clip.write_videofile(avi_file, codec='rawvideo')

# MP4形式の入力ファイルとAVI形式の出力ファイルのパスを指定
input_mp4_file = r"C:\Users\Hikaru Shishido\Videos\Captures\qq.mp4"
output_avi_file = 'output.avi'

# MP4形式の動画をAVI形式に変換
convert_mp4_to_avi(input_mp4_file, output_avi_file)

##### AVI to JPG

In [ ]:
input_filename = r"Image\rec-1.avi"
extract_frames_from_avi(input_filename)

##### JPG to AVI

In [ ]:
def binarize_image(image, threshold=150):
    gray_image = image.convert("L")
    binary_image = gray_image.point(lambda p: p > threshold and 255)
    return binary_image

def create_video_from_images(image_folder, output_video_filename, threshold=150):
    image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(".jpg")]
    image_files.sort()

    if not image_files:
        print("No JPEG images found in the folder.")
        return

    first_image = Image.open(os.path.join(image_folder, image_files[0]))
    frame_size = first_image.size
    frame_rate = 38  # フレームレート

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_filename, fourcc, frame_rate, frame_size)

    for image_file in image_files:
        image_path = os.path.join(image_folder, image_file)
        image = Image.open(image_path)
        binary_image = binarize_image(image, threshold)
        binary_image_np = cv2.cvtColor(
            cv2.cvtColor(np.array(binary_image), cv2.COLOR_GRAY2BGR), cv2.COLOR_BGR2RGB
        )
        out.write(binary_image_np)

    out.release()
    print("Finished")

if __name__ == "__main__":
    image_folder = r"log\20230817172841"  # 画像ファイルが保存されているフォルダ名
    output_video_filename = "output.avi"  # 出力動画ファイル名
    threshold = 150  # 二値化の閾値

    create_video_from_images(image_folder, output_video_filename, threshold)

##### 画像の輝度値をCSVファイルに保存
###### https://imagingsolution.net/program/python/pillow/image_save_load_csv/

In [ ]:
img = Image.open("Image\elegans1.jpg")

img = img.convert("L")
width, height = img.size

data = list(img.getdata())

with open('./ImageProcessing/image_data.csv', 'w', newline='') as csvfile:
    spamwriter  = csv.writer(csvfile)
    x = 0
    for y in range(height):
        line_data = data[x:x+width]
        spamwriter.writerow(line_data)
        x += width

##### CSV to EXCEL

In [ ]:
#Excelファイルを作成
book = excel.Workbook()
#activeなsheetを選択
sheet = book.active
#CSVファイルを指定
csv_filename = r'C:\Users\hideh\OneDrive\Desktop\stage\log\20220921154843\log.csv'

#CSVファイルを開く
with open(csv_filename,newline="") as csvf:
    #CSVファイルからデータを読み込む
    data = csv.reader(csvf)
    
    r = 1
    for line in data:
        if r > 1:
            for i,v in enumerate(line):
                if i >= 0:
                    line[i] = float(v)
        c = 1
        for v in line:
            sheet.cell(row=r,column=c).value=v
            c += 1
        r += 1

#現在時刻を取得
now = datetime.datetime.now()
#ファイルを保存する
book.save(os.getcwd() + "/log/" + "{}.xlsx".format(now.strftime("%Y%m%d%H%M%S")))
#ファイルを閉じる
book.close()